In [1]:
import re
import os
import pandas as pd

regex = re.compile(r'\[.+\]\n', re.IGNORECASE)
file_paths, file_names, emotions, audios = [], [], [], []
emotion_map = {'anger': 'angry', 'happiness': 'happy', 'sadness': 'sad', 'fear': 'fear',
              'disgust': 'disgust'}

In [2]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

1.12.1+cu113
0.12.1+cu113
cuda


In [3]:
bundle = torchaudio.pipelines.HUBERT_BASE
extractor = bundle.get_model()
print(extractor.__class__)
print(bundle.sample_rate)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>
16000


In [4]:
import os
os.listdir('../emotiondata/emotion_data')

['anger',
 'session_entries.csv',
 'fear',
 'Tools and Documentation',
 'disgust',
 'sadness',
 'happiness']

In [5]:
folder_list = ['anger', 'disgust', 'fear', 'happiness', 'sadness']
entries = []
for folder in folder_list:
    cur_file_list = os.listdir(f'../emotiondata/emotion_data/{folder}')
    for i in cur_file_list:
        if i == 's05 (3).wav':
            print("found")
            continue
        entries.append(i)


found


In [6]:
import random
random.shuffle(entries)
session = []
equal_parts = (len(entries)-1)//5 # for equally split the entries into 5 parts
count = 0
main_data_path = '../emotiondata/emotion_data'

In [7]:
#### Only Run once
from tqdm import tqdm 
folder_map = {'a':'anger', 'd':'disgust', 'f':'fear', 'h':'happiness', 's':'sadness'}

file_paths = []
file_names = []
emotions = []
# audios = []
# labels = []


for i in tqdm(range(len(entries))):
    entry = entries[i]
    if "wav" not in entry:
        continue
    folder = folder_map[entry[0]]
    file_path = f'../emotiondata/emotion_data/{folder}/{entry}'
    emotion = emotion_map[folder]
    file_paths.append(file_path)
    file_names.append(entry)
    emotions.append(emotion)

    # assign session to it
    part = (count//equal_parts)%6 + 1
    if part == 6:
        part = 5
    session.append(part)
    count += 1


file = pd.DataFrame({'path':file_paths, 'name': file_names, 'emotion': emotions, 'session': session})
dataframe_path = main_data_path + '/session_entries_hubert.csv'
file.to_csv(dataframe_path)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 604/604 [00:00<00:00, 811714.07it/s]


In [8]:
os.listdir('../emotiondata/emotion_data')

['anger',
 'session_entries.csv',
 'session_entries_hubert.csv',
 'fear',
 'Tools and Documentation',
 'disgust',
 'sadness',
 'happiness']

# Extract Features using Models

In [9]:
dataframe_path = main_data_path + '/session_entries_hubert.csv'
file = pd.read_csv(dataframe_path)[['path', 'name', 'emotion', 'session']]
file.head()

,path,name,emotion,session
0,../emotiondata/emotion_data/anger/a01 (1).wav,a01 (1).wav,angry,1
1,../emotiondata/emotion_data/sadness/s16 (4).wav,s16 (4).wav,sad,1
2,../emotiondata/emotion_data/happiness/h08 (3).wav,h08 (3).wav,happy,1
3,../emotiondata/emotion_data/happiness/h17 (1).wav,h17 (1).wav,happy,1
4,../emotiondata/emotion_data/sadness/s02 (5)b.wav,s02 (5)b.wav,sad,1


In [10]:
from tqdm import tqdm
audios = []
to_be_discarded = []
discarded_name = []
for i in tqdm(range(len(file['path']))):
    path = file['path'][i]
    wave, sr = torchaudio.load(path)
    if sr != bundle.sample_rate:
        wave = torchaudio.functional.resample(wave, sr, bundle.sample_rate)
    with torch.inference_mode():
        feature, _ = extractor.extract_features(wave)
    
    feature = [f[0] for f in feature]
    audio = torch.stack(feature)
    audios.append(audio)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 604/604 [02:26<00:00,  4.13it/s]


# Load Data

In [50]:
holdout = 5
train = file[file['session'] != holdout]
train_audios = [audios[i] for i in range(len(audios)) if file['session'][i] != holdout]
test = file[file['session'] == holdout]
test_audios = [audios[i] for i in range(len(audios)) if file['session'][i] == holdout]

In [51]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, audios, labels, label_transform):
        super(MyDataSet).__init__()
        self.audios = audios
        self.labels = labels
        self.label_transform = label_transform
        
    def __getitem__(self, idx):
        label = self.label_transform[self.labels[idx]]
        audio = self.audios[idx]
        length = audio.size(1)
        return audio, length, label
    
    def __len__(self):
        return len(self.labels)

In [52]:
def collate_indic(data):
    audios, lengths, labels = zip(*data)
    max_len = max(lengths)
    n_ftrs = audios[0].size(2)
    n_dims = audios[0].size(0)
    features = torch.zeros((len(audios), n_dims, max_len, n_ftrs))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
        j, k = audios[i].size(1), audios[i].size(2)
        features[i] = torch.cat([audios[i], torch.zeros((n_dims, max_len - j, k))], dim=1)

    return features, lengths, labels

In [53]:
categories = ['neutral', 'angry', 'happy', 'sad', 'fear', 'disgust']
cate_dic = {}
for i, cate in enumerate(categories):
    cate_dic[cate] = i
cate_dic

{'neutral': 0, 'angry': 1, 'happy': 2, 'sad': 3, 'fear': 4, 'disgust': 5}

In [54]:
from torch.utils.data import DataLoader

train_dataset = MyDataSet(train_audios, train['emotion'].tolist(), cate_dic)
trainloader_args = dict(batch_size=16, shuffle=True)
train_dataloader = DataLoader(train_dataset, **trainloader_args, 
                              collate_fn=collate_indic)

test_dataset = MyDataSet(test_audios, test['emotion'].tolist(), cate_dic)
testloader_args = dict(batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, **testloader_args, 
                             collate_fn=collate_indic)

# 3CNN+LSTM

In [55]:
import torch.nn as nn
import torch.nn.functional as F

class ICASSP3CNN(nn.Module):
    def __init__(self, vocab_size, dims = 12, embed_size=128, hidden_size=512, num_lstm_layers = 2, bidirectional = False, label_size=7):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional
        
        self.aggr = nn.Conv1d(in_channels=dims, out_channels=1, kernel_size=1)
        
        self.embed = nn.Linear(in_features = vocab_size, out_features = embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(3 * embed_size)

        self.lstm = nn.LSTM(input_size = 3 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        n, d, b, t = x.size(0), x.size(1), x.size(2), x.size(3)
        x = torch.flatten(x, start_dim=2)
        input = self.aggr(x)
        input = torch.reshape(input, (n, b, t))
        input = self.embed(input)

        batch_size = input.size(0)
        input = input.transpose(1,2)    # (B,T,H) -> (B,H,T)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True, enforce_sorted=False)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]

        logits = self.linear(h_n)

        return logits


# Train Each Layer

In [56]:
from tqdm import tqdm
from torchsummary import summary
import torch.optim as optim

model = ICASSP3CNN(768)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

for epoch in tqdm(range(epochs)):
    train_loss = 0
    acc_cnt = 0
    err_cnt = 0
    batch_cnt = 0
    model.train()
    for batch, (x, length, y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x, length)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().item()

        #model outputs
        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
        batch_cnt += 1
    
    train_loss = train_loss/batch_cnt
    train_accuracy = acc_cnt/(acc_cnt+err_cnt)
    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)
    
    print(f"epoch:{epoch+1}, train accu:{train_accuracy:.4f},", f"train loss:{train_loss:.2f}")

  2%|█████▊                                                                                                                                                                                                                                                                                          | 1/50 [00:04<03:53,  4.76s/it]

epoch:1, train accu:0.3833, train loss:1.44


  4%|███████████▌                                                                                                                                                                                                                                                                                    | 2/50 [00:09<03:50,  4.79s/it]

epoch:2, train accu:0.6438, train loss:0.92


  6%|█████████████████▎                                                                                                                                                                                                                                                                              | 3/50 [00:14<03:44,  4.79s/it]

...

epoch:48, train accu:1.0000, train loss:0.01


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 49/50 [03:57<00:04,  4.78s/it]

epoch:49, train accu:1.0000, train loss:0.00


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:02<00:00,  4.85s/it]

epoch:50, train accu:1.0000, train loss:0.00


# Model Test

In [57]:
test_loss = 0
acc_cnt = 0
err_cnt = 0
batch_cnt = 0
model.eval()

for x, lengths, y in test_dataloader:

    x = x.to(device)
    y = y.to(device)

    logits = model(x, lengths)
    loss = criterion(logits, y)
    test_loss += loss.cpu().item()

    out_val, out_indices = torch.max(logits, dim=1)
    tar_indices = y

    for i in range(len(out_indices)):
        if out_indices[i] == tar_indices[i]:
            acc_cnt += 1
        else:
            err_cnt += 1
    batch_cnt += 1

test_loss = test_loss/batch_cnt
test_accuracy = acc_cnt/(acc_cnt+err_cnt)
print(f'test accuracy: {test_accuracy}')

test accuracy: 0.7983870967741935


In [58]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

aggr.weight tensor([[[ 0.0753],
         [ 0.2551],
         [ 0.1802],
         [-0.2935],
         [-0.2755],
         [ 0.2087],
         [ 0.0563],
         [ 0.1467],
         [-0.0244],
         [-0.2291],
         [ 0.2658],
         [-0.1543]]], device='cuda:0')
aggr.bias tensor([0.1089], device='cuda:0')
embed.weight tensor([[ 3.2320e-03,  1.0965e-02, -2.3358e-04,  ..., -1.3281e-02,
         -3.1011e-02,  1.6737e-02],
        [ 1.1983e-02, -1.1725e-02, -3.3963e-02,  ...,  2.8910e-02,
         -1.2704e-02,  6.3700e-02],
        [ 2.1200e-02,  2.2522e-02,  1.1867e-02,  ..., -1.0812e-02,
         -6.4397e-03,  2.9728e-05],
        ...,
        [-6.4769e-02, -2.6768e-02, -1.7272e-02,  ..., -1.8479e-02,
          5.4517e-02,  9.3301e-04],
        [ 3.7812e-02, -3.9618e-02, -1.8583e-02,  ...,  5.4425e-02,
         -3.8337e-02,  1.0364e-02],
        [-2.0452e-02, -1.7647e-02, -2.9176e-02,  ..., -4.6143e-02,
          1.5949e-02,  5.5211e-02]], device='cuda:0')
embed.bias tensor([-0.0